In [11]:
#!/usr/bin/env python

import pandas as pd
import numpy as np
from collections import Counter
import warnings

warnings.filterwarnings('ignore')
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    #numCounter = {}
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        headerTable = dict(Counter(headerTable) + Counter(trans))
    #headerTableCopy = headerTable.copy()

    freqItemCounter = {k: v for k, v in headerTable.items() if v >= minSup}

    freqItemSet = set(freqItemCounter.keys())

    #print 'freqItemSet: ',freqItemSet

    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out

    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

def loadData():
    data = pd.read_csv('../FP_data/retail.csv',sep=',',warn_bad_lines=False,error_bad_lines=False)
    data = data.sample(n=100).reset_index(drop=True)
    data = data.values
    new_data = []
    for i in range(np.shape(data)[0]):
        new_data.append(data[i][~np.isnan(data[i])])
    return new_data

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print('conditional tree for: ',newFreqSet)
            #myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)
            
if __name__ == '__main__':
    minSup = 2
    data = loadData()
    initSet = createInitSet(data)
    myFPtree, myHeaderTab = createTree(initSet, minSup)
    myFPtree.disp()
    myFreqList = []
    mineTree(myFPtree, myHeaderTab, minSup, set([]), myFreqList)
    print('Well done')

   Null Set   1
     39.0   62
       48.0   40
         38.0   10
           161.0   1
             2437.0   1
               47.0   1
                 371.0   1
           1218.0   1
           41.0   4
             544.0   1
               9191.0   1
                 790.0   1
                   475.0   1
             2792.0   1
               11.0   1
                 31.0   1
             2958.0   1
               3476.0   1
                 36.0   1
                   449.0   1
                     10571.0   1
             147.0   1
               426.0   1
                 47.0   1
                   319.0   1
                     1218.0   1
                       10571.0   1
                         3553.0   1
                           1277.0   1
                             382.0   1
           1393.0   1
             2624.0   1
               824.0   1
                 4220.0   1
           286.0   1
           110.0   1
           1214.0   1
             286.0   1
         

In [13]:
#!/usr/bin/env python
import pandas as pd
import numpy as np


def loadData():
    data = pd.read_csv('../Kmeans数据集/Sales_Transactions_Dataset_Weekly.csv')
    newData = data[data.columns[55:]]
    labelData = pd.DataFrame(data['Product_Code'])
    #new_data = pd.concat([data[data.columns[55:]],data['Product_Code']],axis=1)
    return newData, labelData

def distEclud(vecA, vecB):
    return np.sqrt(sum(np.power(list(map(lambda x: x[0]-x[1], zip(vecA, vecB))), 2))) #la.norm(vecA-vecB)

def randCent(dataSet, k):
    m = dataSet.shape[1]
    centroids = np.mat(np.random.random((k,m)))#create centroid mat
    return centroids

def kMeans(dataSet, labelData, k, distMeas=distEclud, createCent=randCent):
    m = dataSet.shape[0] #样本数
    clusterAssment = np.mat(np.zeros((m,2))) #m*2的矩阵
    clusterAssment = pd.DataFrame(clusterAssment)
    clusterAssment.columns = ['point','dist']
    centroids = createCent(dataSet, k) #初始化k个中心
    clusterChanged = True
    while clusterChanged:      #当聚类不再变化
        clusterChanged = False
        for i in range(m):
            minDist = np.inf; minLabel = -1
            for j in range(k): #找到最近的质心
                distJI = distMeas(centroids[j,:].tolist()[0],dataSet.loc[i].tolist())
                if distJI < minDist:
                    minDist = distJI
                    minLabel = j
            if clusterAssment.iloc[i,0] != minLabel:
                clusterChanged = True
            # 第1列为所属质心，第2列为距离
            clusterAssment.loc[i,:] = minLabel,minDist**2
        #print(centroids)
        # 更改质心位置
        for cent in range(k):
            ptsInClust = dataSet[clusterAssment['point']==cent]
            centroids[cent,:] = np.mean(ptsInClust, axis=0)
    return centroids, clusterAssment



if __name__ == '__main__':
    k = 10
    dataSet, labelData = loadData()
    centroids, clusterAssment = kMeans(dataSet, labelData, k, distMeas=distEclud, createCent=randCent)
    print(clusterAssment)


     point      dist
0      4.0  2.135655
1      2.0  2.507844
2      5.0  3.948026
3      0.0  2.431614
4      0.0  2.180291
5      9.0  2.420109
6      2.0  2.187374
7      4.0  2.766835
8      4.0  2.532766
9      4.0  1.939960
10     4.0  2.805989
11     4.0  2.274907
12     5.0  1.921003
13     9.0  1.401823
14     5.0  1.943853
15     3.0  1.753892
16     5.0  1.780356
17     3.0  2.065607
18     3.0  2.134785
19     4.0  1.681732
20     3.0  2.436182
21     5.0  2.110112
22     5.0  1.929387
23     5.0  1.679108
24     3.0  1.731958
25     9.0  1.928009
26     3.0  2.531954
27     5.0  2.398515
28     2.0  1.378491
29     5.0  2.315403
..     ...       ...
781    0.0  1.912343
782    0.0  2.216438
783    0.0  5.863897
784    7.0  2.441446
785    6.0  1.847007
786    6.0  2.833351
787    0.0  1.944035
788    9.0  1.908900
789    0.0  3.045302
790    9.0  1.757310
791    7.0  2.446926
792    7.0  1.012692
793    6.0  2.815757
794    9.0  2.155740
795    9.0  2.581879
796    0.0  2